<a href="https://colab.research.google.com/github/PawanKalyanJada/NMT/blob/main/machine_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
tpu_strategy = tf.distribute.TPUStrategy(tpu)

INFO:tensorflow:Initializing the TPU system: grpc://10.126.53.26:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.126.53.26:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
l=[]
with open('/content/drive/MyDrive/english.txt','r') as f:
  l=f.readlines()

In [ ]:
with open('/content/drive/MyDrive/telugu.txt') as f:
  l1=f.readlines()

In [ ]:
for i in range(len(l1)):
  if('right' in l1[i]):
    p=l1[i].index('|')
    k=l1[i][p+1:].index('|')
    l1[i]=l1[i][k+1:]
  if('left' in l1[i]):
    p=l1[i].index('|')
    k=l1[i][p+1:].index('|')
    l1[i]=l1[i][k+1:]

In [ ]:
for i in range(len(l)):
  if(l[i][-2]=='.'):
    if(l1[i][-2]!='.'):
      l1[i]=l1[i][:-1]+'.'+l1[i][-1]
  else:
    if(l1[i][-2]=='.'):
      l1[i]=l1[i][:-2]+l1[i][-1]

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from nltk.tokenize import word_tokenize
word_tokenize(l[678])

['origins', '.']

In [ ]:
word_tokenize(l1[678])

['మూలాలు', '.']

In [ ]:
e=[]
for i in l:
  i=i.lower()
  p=word_tokenize(i)
  e.append(p)
t=[]
for i in l1:
  p=word_tokenize(i)
  t.append(p)

In [ ]:
d={}
d['<unk>']=1
c=2
for i in e:
  for j in i:
    if(not(j in d)):
      d[j]=c
      c+=1

In [ ]:
eng_vocab=len(d)+1

In [ ]:
d1={}
d1['<unk>']=1
d1['start']=2
d1['end']=3
c=4
for i in t:
  for j in i:
    if(not(j in d1)):
      d1[j]=c
      c+=1

In [ ]:
x=[]
for i in e:
  p=[]
  for j in i:
    p.append(d[j])
  if(len(p)>128):
    p=p[:128]
  x.append(p)

In [ ]:
for i in range(len(x)):
  x[i]=x[i]+[0]*(128-len(x[i]))

In [ ]:
import numpy as np
x=np.array(x)

In [ ]:
y=[]
for i in t:
  p=[]
  for j in i:
    p.append(d1[j])
  if(len(p)>127):
    p=p[:127]
  p.append(3)
  y.append(p)

In [ ]:
for i in range(len(y)):
  y[i]=y[i]+[0]*(128-len(y[i]))

In [ ]:
y=np.array(y)

In [ ]:
yp=[]
for i in t:
  p=[2]
  for j in i:
    p.append(d1[j])
  if(len(p)>127):
    p=p[:127]
  p.append(3)
  yp.append(p)

In [ ]:
for i in range(len(yp)):
  yp[i]=yp[i]+[0]*(128-len(yp[i]))

In [ ]:
yp=np.array(yp)

In [ ]:
yp[0]

array([2, 4, 5, 6, 7, 8, 9, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
y[0]

array([4, 5, 6, 7, 8, 9, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
from tensorflow.keras.models import Model
from keras.layers import Embedding,Dense,Bidirectional,Input,LSTM

In [ ]:
tel_vocab=len(d1)+1

In [ ]:
len(d1)

37632

In [ ]:
with tpu_strategy.scope(): 
  encoder_inputs=Input((128,))
  embedd=Embedding(eng_vocab,100)
  encoder_inputs1=embedd(encoder_inputs)
  encoder_lstm=(LSTM(64,return_state=True))
  encoder_outputs,state_h,state_c=encoder_lstm(encoder_inputs1)
  encoder_states=[state_h,state_c]
  decoder_inputs=Input(shape=(None,))
  dec_embed=Embedding(tel_vocab,100)
  decoder_inputs1=dec_embed(decoder_inputs)
  decoder_lstm=(LSTM(64,return_state=True,return_sequences=True))
  decoder_outputs,_,_=decoder_lstm(decoder_inputs1,initial_state=encoder_states)
  decoder_dense=Dense(tel_vocab,activation='softmax')
  decoder_outputs=decoder_dense(decoder_outputs)
  model=Model([encoder_inputs, decoder_inputs],decoder_outputs)
  encoder_model = Model(encoder_inputs, encoder_states)
  decoder_state_input_h = Input(shape=(64,))
  decoder_state_input_c = Input(shape=(64,))
  decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
  dec_emb2=dec_embed(decoder_inputs)
  decoder_outputs, state_h, state_c = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
  decoder_states = [state_h, state_c]
  decoder_outputs = decoder_dense(decoder_outputs)
  decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [ ]:
import keras
with tpu_strategy.scope():
  model.compile(optimizer=keras.optimizers.Adam(),loss='sparse_categorical_crossentropy',metrics=['accuracy'],steps_per_execution=32)

In [ ]:
model.fit([x,yp],y,epochs=15,batch_size=128)

Epoch 1/15
337/337 [==============================] - 31s 93ms/step - loss: 2.1743 - accuracy: 0.9071
Epoch 2/15
337/337 [==============================] - 14s 40ms/step - loss: 0.7634 - accuracy: 0.9170
Epoch 3/15
337/337 [==============================] - 14s 40ms/step - loss: 0.7346 - accuracy: 0.9175
Epoch 4/15
337/337 [==============================] - 14s 41ms/step - loss: 0.7154 - accuracy: 0.9188
Epoch 5/15
337/337 [==============================] - 14s 40ms/step - loss: 0.6972 - accuracy: 0.9202
Epoch 6/15
337/337 [==============================] - 14s 40ms/step - loss: 0.6789 - accuracy: 0.9211
Epoch 7/15
337/337 [==============================] - 14s 40ms/step - loss: 0.6598 - accuracy: 0.9220
Epoch 8/15
337/337 [==============================] - 14s 40ms/step - loss: 0.6394 - accuracy: 0.9226
Epoch 9/15
337/337 [==============================] - 14s 40ms/step - loss: 0.6179 - accuracy: 0.9234
Epoch 10/15
337/337 [==============================] - 14s 40ms/step - loss: 0.596

In [ ]:
model.fit([x,yp],y,epochs=20,batch_size=128)

Epoch 1/20
337/337 [==============================] - 14s 41ms/step - loss: 0.4644 - accuracy: 0.9302
Epoch 2/20
337/337 [==============================] - 14s 41ms/step - loss: 0.4438 - accuracy: 0.9320
Epoch 3/20
337/337 [==============================] - 14s 40ms/step - loss: 0.4235 - accuracy: 0.9342
Epoch 4/20
337/337 [==============================] - 14s 40ms/step - loss: 0.4035 - accuracy: 0.9371
Epoch 5/20
337/337 [==============================] - 14s 40ms/step - loss: 0.3834 - accuracy: 0.9404
Epoch 6/20
337/337 [==============================] - 14s 40ms/step - loss: 0.3641 - accuracy: 0.9439
Epoch 7/20
337/337 [==============================] - 14s 41ms/step - loss: 0.3456 - accuracy: 0.9475
Epoch 8/20
337/337 [==============================] - 14s 41ms/step - loss: 0.3254 - accuracy: 0.9520
Epoch 9/20
337/337 [==============================] - 14s 40ms/step - loss: 0.3062 - accuracy: 0.9557
Epoch 10/20
337/337 [==============================] - 14s 40ms/step - loss: 0.288

In [ ]:
model.fit([x,yp],y,epochs=10,batch_size=128)

Epoch 1/10
337/337 [==============================] - 14s 41ms/step - loss: 0.0628 - accuracy: 0.9885
Epoch 2/10
337/337 [==============================] - 14s 40ms/step - loss: 0.0622 - accuracy: 0.9886
Epoch 3/10
337/337 [==============================] - 14s 40ms/step - loss: 0.0620 - accuracy: 0.9886
Epoch 4/10
337/337 [==============================] - 14s 40ms/step - loss: 0.0598 - accuracy: 0.9889
Epoch 5/10
337/337 [==============================] - 14s 40ms/step - loss: 0.0586 - accuracy: 0.9891
Epoch 6/10
337/337 [==============================] - 14s 41ms/step - loss: 0.0579 - accuracy: 0.9892
Epoch 7/10
337/337 [==============================] - 14s 40ms/step - loss: 0.0570 - accuracy: 0.9893
Epoch 8/10
337/337 [==============================] - 14s 40ms/step - loss: 0.0560 - accuracy: 0.9895
Epoch 9/10
337/337 [==============================] - 14s 40ms/step - loss: 0.0549 - accuracy: 0.9896
Epoch 10/10
337/337 [==============================] - 14s 40ms/step - loss: 0.054

In [ ]:
r=encoder_model.predict(np.array([x[500]]))

In [ ]:
p=np.array([[5967]])

In [ ]:
r1=decoder_model.predict([p]+r)

In [ ]:
a,b,c=r1

In [ ]:
np.argmax(a[0,0,:]).reshape(1,1)

array([[10]])

In [ ]:
td1={a:b for b,a in d1.items()}

In [ ]:
td1[0]='pad'

In [ ]:
td1[2]

'start'

In [ ]:
def predict_sequence(infenc, infdec, source, n_steps):
  state = infenc.predict(source)
  target_seq = np.array([[2]])
  output = list()
  for t in range(n_steps):
    yhat, h, c= infdec.predict([target_seq] + state)
    state = [h, c]
    k=np.argmax(yhat)
    target_seq = k.reshape(1,1)
    s=td1[k]
    print(s)
    output.append(s)
  return output

In [ ]:
s="lets see how it goes"
s=word_tokenize(s)
s1=[]
for i in s:
  if(i in d):
    s1.append(d[i])
  else:
    s1.append(d['<unk>'])
s1=s1+[0]*(128-len(s1))
s1=np.array(s1)

In [ ]:
res=predict_sequence(encoder_model,decoder_model,np.array([s1]),10)

కాని
1922
లో
ఉత్తరప్రదేశ్
చౌరీచౌరా
లో
ఉద్రేకాలు
పెల్లుబికి
హింస
చెలరేగింది
